<img src="images/tf_a3c.png" align=right width=40%></img>
# TensorFlow Implementation of A3C
Author: Jin Yeom (jinyeom@utexas.edu)

## Contents


In [2]:
import gym
import tensorflow as tf

In this notebook, we implement the **Asynchronous Advantage Actor-Critic (A3C)** algorithm with **[TensorFlow](https://www.tensorflow.org/)**. The focus of this project is to (1) learn how to work with TensorFlow in more depth, and (2) have my own working implementation of A3C algorithm for other current or future reinforcement learning projects. For the simplicity of this notebook, we will aim to build an agent to play an Atari 2600 game, **Pong**. 

Note that **A2C** (synchronous variation of A3C) has been shown to be more cost effective when trained on a single GPU, or TPU, for that matter. We will first try our A3C implementation on a *multi-core CPU* (most likely on GCP), then A2C on a *TPU (Tensor Processing Unit)* to compare their costs and performances.